In [1]:
import ROOT
import numpy as np
import# Models pandas as pd 
import json

ROOT.gROOT.ProcessLine( "gErrorIgnoreLevel = kError;");

Welcome to JupyROOT 6.29/01


# Models

## Malloc

In [3]:
filename = "results.json" 
with open(filename, "r") as f:
    jdata = json.load(f)
df = pd.DataFrame.from_dict(jdata["benchmarks"]).astype({"real_time" : float})
df["param"] = df["name"].apply(lambda s: float(s.split("/")[1]))
df["func_name"] = df["name"].apply(lambda s: s.split("/")[0][3:])
df

FileNotFoundError: [Errno 2] No such file or directory: 'results.json'